# 11. Real-world problems and additional topics

During this course we have either used simple demo examples designed to illustrate specific aspects of Pandas, or real-world data where we ignored some of the potentials problems. In this notebook, we address a few of the common problems that you might encounter. We also address some other common situations that you might encounter such as dealing with specific formats like dates.

In [4]:
import pandas as pd
import numpy as np

## Absent values

Very often, data that you need are imperfect: they migth have missing values or some values might be "incorrect", e.g. in a column supposed to contain numbers you suddently encounter text. Pandas offers many ways to fix these problems.

Let's once again import our penguin data:

In [5]:
penguins = pd.read_csv('https://raw.githubusercontent.com/allisonhorst/palmerpenguins/master/inst/extdata/penguins.csv')

In [6]:
penguins.shape

(344, 8)

In [7]:
penguins.head(5)

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,male,2007
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,female,2007
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,female,2007
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN,2007
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,female,2007


Here we see that some values are missing and filled with ```NaN```. As we have seen previously, Pandas is capable of handling ```NaN``` values e.g. when calculating statistics. But sometimes we want to remove those values. We can for example use ```.dropna()``` for this:

In [8]:
penguins_clean = penguins.dropna()

In [9]:
penguins_clean.head(5)

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,male,2007
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,female,2007
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,female,2007
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,female,2007
5,Adelie,Torgersen,39.3,20.6,190.0,3650.0,male,2007


We see that for example the index 3 has disappeared. ```.dropna()``` drops each line that has an ```NaN``` in any column.

You can also be more precise and check for ```NaN``` in a given column:

In [10]:
penguins.bill_length_mm.isna()

0      False
1      False
2      False
3       True
4      False
       ...  
339    False
340    False
341    False
342    False
343    False
Name: bill_length_mm, Length: 344, dtype: bool

Using indexing, we can now remove **only lines** which have ```NA``` in that column:

In [11]:
penguins_clean2 = penguins[~penguins.bill_length_mm.isna()]
penguins_clean2.head(5)

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,male,2007
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,female,2007
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,female,2007
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,female,2007
5,Adelie,Torgersen,39.3,20.6,190.0,3650.0,male,2007


## Replacing *bad* values

When importing a table, Pandas is capable of recognizing specific values as being ```NaN```. For example the text NaN or missing values are turned in "offficial" ```NaN```. But often there are just mistakes in the table. For example:

In [12]:
composer_url = 'https://github.com/guiwitz/NumpyPandas_course/blob/master/Data/composers.xlsx?raw=true'
composers = pd.read_excel(composer_url, sheet_name=1)

In [13]:
composers

,composer,birth,death,city
0,Mahler,1860.0,1911,Kaliste
1,Beethoven,1770.0,1827,Bonn
2,Puccini,1858.0,1924,Lucques
3,Shostakovich,1906.0,1975,Saint-Petersburg
4,Sibelius,10.0,unknown,unknown
5,Haydn,NaN,NaN,Röhrau


Here we see that we have both ```NaN``` values and other "bad" values in the form of ```uknown``` text. This is a problem because now the columns containing that text are not "number-columns" anymore and for example we can't do operations:

In [14]:
np.log(composers['death'])

TypeError: loop of ufunc does not support argument 0 of type int which has no callable log method

We therefore have first to clean-up the table. One solution is to replace all bade values. For that we can use ```.replace```:

In [15]:
composers_clean = composers.replace('unknown', np.nan)
composers_clean

,composer,birth,death,city
0,Mahler,1860.0,1911.0,Kaliste
1,Beethoven,1770.0,1827.0,Bonn
2,Puccini,1858.0,1924.0,Lucques
3,Shostakovich,1906.0,1975.0,Saint-Petersburg
4,Sibelius,10.0,NaN,NaN
5,Haydn,NaN,NaN,Röhrau


In [16]:
np.log(composers_clean['death'])

0    7.555382
1    7.510431
2    7.562162
3    7.588324
4         NaN
5         NaN
Name: death, dtype: float64

Alternatively we can fix this problem at import time as well because we might have a lsit of all "bad entries". For that we can use the option

In [17]:
pd.read_excel(composer_url, na_values=['unknown'], sheet_name=1)

,composer,birth,death,city
0,Mahler,1860.0,1911.0,Kaliste
1,Beethoven,1770.0,1827.0,Bonn
2,Puccini,1858.0,1924.0,Lucques
3,Shostakovich,1906.0,1975.0,Saint-Petersburg
4,Sibelius,10.0,NaN,NaN
5,Haydn,NaN,NaN,Röhrau


## Imputation

Once "bad" values have been replaced by n.a values we can simply drop them. To avoid loosing too much data in case many single items in different columns have missing values, we can try to replace those values by actual numbers. For example in a numerical column we can replace missing values by the median or mean of the existing values. While we could do that "by hand", there are ways to automate this action know as imputation.

Here we give a preview of the scikit-learn library that we will use later for modelling and see already now how we can use it for imputation.

First we load an object that will take care of imputation from the ```impute``` module of scikit-learn:

In [18]:
from sklearn.impute import SimpleImputer

Then we instantiate an imputer and say which values we want to replace (here n.a) and how we want this to be done e.g. with the mean of the other values:

In [19]:
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')

Then we "fit" the imputer i.e. we compute the actual mean of the column we want to fix. For this we pass an array or a dataframe e.g. the ```birth``` and ```death``` columns:

In [25]:
composers_clean

,composer,birth,death,city
0,Mahler,1860.0,1911.0,Kaliste
1,Beethoven,1770.0,1827.0,Bonn
2,Puccini,1858.0,1924.0,Lucques
3,Shostakovich,1906.0,1975.0,Saint-Petersburg
4,Sibelius,10.0,NaN,NaN
5,Haydn,NaN,NaN,Röhrau


In [24]:
imputer.fit(composers_clean[['birth', 'death']])

SimpleImputer()

This object contains various infos. Among other things, the values that can now be used to replace missing values:

In [27]:
imputer.statistics_

array([1480.8 , 1909.25])

To actually replace the missing values we now use the ```transform``` method:

In [29]:
imputer.transform(composers_clean[['birth', 'death']])

array([[1860.  , 1911.  ],
       [1770.  , 1827.  ],
       [1858.  , 1924.  ],
       [1906.  , 1975.  ],
       [  10.  , 1909.25],
       [1480.8 , 1909.25]])

One should be very careful with imputation as it can introduce severe bias in the data. In the above example we see that the "bad" values of 10 for the birth date completely skews the imputation. Here a median would probably be better. We show it here by using directly a combination of fit and transform:

In [31]:
imputer2 = SimpleImputer(missing_values=np.nan, strategy='median')

In [32]:
imputer2.fit_transform(composers_clean[['birth', 'death']])

array([[1860. , 1911. ],
       [1770. , 1827. ],
       [1858. , 1924. ],
       [1906. , 1975. ],
       [  10. , 1917.5],
       [1858. , 1917.5]])

## Dealing with time

Pandas offers many possibilities to help dealing with data containing time, e.g. to calculate how many days have passed between two dates.

In [15]:
composers = pd.read_excel(composer_url)

In [16]:
composers

,composer,birth,death,city
0,Mahler,1860,1911,Kaliste
1,Beethoven,1770,1827,Bonn
2,Puccini,1858,1924,Lucques
3,Shostakovich,1906,1975,Saint-Petersburg


We transform now the ```birth``` and ```death``` columns into ```datetime``` objects using ```pd.to_datetime``` and by specifying a format, here just a year ```%Y```:

In [17]:
composers['birth'] = pd.to_datetime(composers['birth'], format = '%Y')
composers['death'] = pd.to_datetime(composers['death'], format = '%Y')
composers

,composer,birth,death,city
0,Mahler,1860-01-01,1911-01-01,Kaliste
1,Beethoven,1770-01-01,1827-01-01,Bonn
2,Puccini,1858-01-01,1924-01-01,Lucques
3,Shostakovich,1906-01-01,1975-01-01,Saint-Petersburg


You can see that the year has been turned into an actual date (using 1st January as default). We create now an age column:

In [18]:
composers['age'] = (composers.death - composers.birth)
composers

,composer,birth,death,city,age
0,Mahler,1860-01-01,1911-01-01,Kaliste,18627 days
1,Beethoven,1770-01-01,1827-01-01,Bonn,20818 days
2,Puccini,1858-01-01,1924-01-01,Lucques,24105 days
3,Shostakovich,1906-01-01,1975-01-01,Saint-Petersburg,25202 days


Since the columns are datetime objects, Pandas know that it should actually compute difference in dates and by defaults gives days. Here the computation is trivial, but if you deal with formats such as ```HH:MM:SS``` this is very useful.